In [1]:
import pandas as pd

In [2]:
import torch

In [4]:
df = pd.read_csv('intern_task.csv')
df

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,3.0,3.0,0.333333,0.0,0.333333,...,0.0,0.000000,0.454545,0.890238,8.655534,1.000000,0.077778,0.002222,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.773976,23.130514,0.000000,0.027826,0.000430,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.000000,0.000000,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.000000,0.000000,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,273.0,79.670665,0.200000,0.990119,31.786048,0.333333,0.046512,0.000307,24.0,8.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235253,2,29995,1.0,0.0,0.0,0.0,1.0,0.500000,0.0,0.000000,...,0.0,0.000000,0.000000,0.471409,0.000000,0.000000,0.001350,0.000002,3.0,1.500000
235254,2,29995,1.0,0.0,1.0,0.0,1.0,0.500000,0.0,0.500000,...,0.0,0.000000,0.000000,0.471409,39.908056,0.000000,0.004850,0.000014,9.0,4.500000
235255,1,29995,1.0,0.0,0.0,0.0,1.0,0.500000,0.0,0.000000,...,0.0,0.000000,0.000000,0.471409,0.000000,0.000000,0.001064,0.000001,1.0,0.500000
235256,2,29995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [5]:
df['rank'].value_counts()

rank
0    121521
1     75815
2     31910
3      4209
4      1803
Name: count, dtype: int64

## Data splitting

In [6]:
# код взял отсюда https://sease.io/2022/07/how-to-split-your-dataset-into-train-and-test-sets-for-evaluating-learning-to-rank-models.html
from sklearn.model_selection import StratifiedGroupKFold
min_diff = len(df)
test_set_size = int(len(df)*0.2)
min_train_idx = []
min_test_idx = []
groups_split_object = StratifiedGroupKFold(n_splits=int(len(df)/test_set_size))
for train_idx, test_idx in groups_split_object.split(df, df['rank'], df['query_id']):
    diff = abs(len(test_idx) - test_set_size)
    if diff < min_diff:
        min_diff = diff
        min_train_idx = train_idx
        min_test_idx = test_idx
        if diff == 0:
            break
interactions_train = df.iloc[min_train_idx]
interactions_test = df.iloc[min_test_idx]

In [7]:
min_diff = len(interactions_test)
test_set_size = int(len(interactions_test)*0.5)
min_train_idx = []
min_test_idx = []
groups_split_object = StratifiedGroupKFold(n_splits=int(len(interactions_test)/test_set_size))
for train_idx, test_idx in groups_split_object.split(interactions_test, interactions_test['rank'], interactions_test['query_id']):
    diff = abs(len(test_idx) - test_set_size)
    if diff < min_diff:
        min_diff = diff
        min_train_idx = train_idx
        min_test_idx = test_idx
        if diff == 0:
            break
interactions_val = interactions_test.iloc[min_train_idx]
interactions_test = interactions_test.iloc[min_test_idx]

In [8]:
X_train, y_train = interactions_train.drop('rank', axis=1), interactions_train['rank']
X_val, y_val = interactions_val.drop('rank', axis=1), interactions_val['rank']
X_test, y_test = interactions_test.drop('rank', axis=1), interactions_test['rank']

In [19]:
from catboost import Pool

train = Pool(
    data=X_train.drop('query_id', axis=1),
    label=y_train,
    group_id=X_train['query_id']
)

val = Pool(
    data=X_val.drop('query_id', axis=1),
    label=y_val,
    group_id=X_val['query_id']
)

test = Pool(
    data=X_test.drop('query_id', axis=1),
    label=y_test,
    group_id=X_test['query_id']
)

## Pairwise learning

In [42]:
from catboost import CatBoostRanker

cb_rank = CatBoostRanker(iterations=2000, loss_function='PairLogit', train_dir='PairLogit', 
                         custom_metric = ['NDCG', 'NDCG:top=5', 'MAP:top=5'],
                         task_type="GPU", silent=True, use_best_model=True)
cb_rank.fit(train, eval_set=val, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because MAP, NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


In [41]:
cb_rank_2 = CatBoostRanker(iterations=2000, loss_function='YetiRank', train_dir='YetiRank', 
                         custom_metric = ['NDCG:top=5', 'MAP:top=5'],
                         task_type="GPU", silent=True, use_best_model=True)
cb_rank_2.fit(train, eval_set=val, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because MAP, NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


In [43]:
cb_rank_3 = CatBoostRanker(iterations=2000, loss_function='PairLogitPairwise', train_dir='PairLogitPairwise', 
                         custom_metric = ['NDCG', 'NDCG:top=5', 'MAP:top=5'],
                         task_type="GPU", silent=True, use_best_model=True)
cb_rank_3.fit(train, eval_set=val, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because MAP, NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


In [45]:
cb_rank_4 = CatBoostRanker(iterations=3000, loss_function='YetiRankPairwise', train_dir='YetiRankPairwise', 
                         custom_metric = ['NDCG:top=5', 'MAP:top=5'],
                         task_type="GPU", silent=True, use_best_model=True)
cb_rank_4.fit(train, eval_set=val, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because MAP, NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:top=5;type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=5 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


In [53]:
best_model = cb_rank_4
metrics = best_model.eval_metrics(test, 
                             metrics=['NDCG', 'NDCG:top=5', 'MAP:top=5'],
                             plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))